# Scrape Trump White House transcript text

> Use the requests library to read scrape remarks transcripts from the Trump [White House archive](https://trumpwhitehouse.archives.gov/remarks/page/1).

---

## Config

#### Python tools and Jupyter settings

In [1]:
%load_ext lab_black

In [2]:
import re
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm, trange

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None

---

## Read data

In [4]:
today = pd.Timestamp("today").strftime("%Y_%m_%d")

#### Read the metadata scraped in `00` notebook

In [5]:
meta_src = pd.read_json("data/processed/trump_wh_remarks_release_metadata.json")
meta_df = meta_src[
    meta_src["headline"].str.contains("Remarks by President Trump")
].drop_duplicates()

In [6]:
meta_df.head()

,headline,url,date_str,page,location,date
0,Remarks by President Trump In Farewell Address to the Nation,https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-farewell-address-nation/,"Jan 19, 2021",1,,2021-01-19
1,"Remarks by President Trump at the 45th Mile of New Border Wall | Reynosa-McAllen, TX",https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-45th-mile-new-border-wall-reynosa-mcallen-tx/,"Jan 12, 2021",1,"Reynosa-McAllen, TX",2021-01-12
2,"Remarks by President Trump Before Air Force One Departure | Joint Base Andrews, MD",https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-air-force-one-departure-joint-base-andrews-md-011221/,"Jan 12, 2021",1,"Joint Base Andrews, MD",2021-01-12
3,Remarks by President Trump Before Marine One Departure,https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-marine-one-departure-011221/,"Jan 12, 2021",1,,2021-01-12
13,Remarks by President Trump at the Operation Warp Speed Vaccine Summit,https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-operation-warp-speed-vaccine-summit/,"Dec 8, 2020",2,,2020-12-08


In [7]:
speech_list = []

for k, v in meta_df.iterrows():
    r = requests.get(v["url"])
    soup = BeautifulSoup(r.text, "html.parser")

    # Find the section with class 'body-content'
    body_content_section = soup.find("div", class_="page-content__content editor")

    # Find all <p> tags within the main content div and concatenate them into a single string
    paragraphs = body_content_section.find_all("p")
    speech_text = " ".join([p.get_text() for p in paragraphs])

    # Create a dictionary with the speech information
    speech_dict = {
        "headline": v["headline"],
        "date": v["date"],
        "url": v["url"],
        "text": speech_text,
    }

    # Append the dictionary to the speech_list
    speech_list.append(speech_dict)

In [ ]:
df = pd.DataFrame(speech_list)

In [12]:
df.head()

headline  \
0                          Remarks by President Trump In Farewell Address to the Nation   
1  Remarks by President Trump at the 45th Mile of New Border Wall | Reynosa-McAllen, TX   
2    Remarks by President Trump Before Air Force One Departure | Joint Base Andrews, MD   
3                                Remarks by President Trump Before Marine One Departure   
4                 Remarks by President Trump at the Operation Warp Speed Vaccine Summit   

        date  \
0 2021-01-19   
1 2021-01-12   
2 2021-01-12   
3 2021-01-12   
4 2020-12-08   

                                                                                                                                       url  \
0                               https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-farewell-address-nation/   
1          https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-45th-mile-new-border-wall-reynosa-mcallen-tx/   
2  https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-air-force-one-departure-joint-base-andrews-md-011221/   
3                           https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-marine-one-departure-011221/   
4                   https://trumpwhitehouse.archives.gov/briefings-statements/remarks-president-trump-operation-warp-speed-vaccine-summit/   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

---

## Export

#### All to CSV & JSON

In [9]:
df.to_csv(f"data/processed/trump_white_house_remarks.csv", index=False)
df.to_json(
    f"data/processed/trump_white_house_remarks.json",
    indent=4,
    orient="records",
)